In [1]:
import os
import pickle

import matplotlib.pyplot as plt

import numpy as np
import pymc3
import theano.tensor as tt

In [2]:
KB = 0.0019872041        # in kcal/mol/K
INJ_VOL = 1.2e-5         # in liter
CELL_CONCENTR = 0.1      # milli molar
SYRINGE_CONCENTR = 1.0   # milli molar

HEAT_FILE = "inputs/Mg1EDTAp1a.DAT"
OUT_DIR = "outputs"

In [5]:
def equilibrium_concentrations(Kd1, Kd2, C0_R, C0_L1, C0_L2, V):
    """
    REF: Zhi-Xin Wang An exact mathematical expression for describing competitive
                        binding of two different ligands to a protein molecule
                        FEBS Letters 360 (1995) 111-114

    Kd1, Kd2        :   disociation constants of ligand 1 and 2, respectively (M)
    C0_R            :   total concentration receptor   (M)
    C0_L1, C0_L2    :   total concentrations of ligand 1 and ligand 2, respectively (M)
    V               :   volume of sample cell (L)
    return
                [RL1, RL2]  complex concentrations (M)
    """

    a = Kd1 + Kd2 + C0_L1 + C0_L2 - C0_R

    b = Kd2*(C0_L1 - C0_R) + Kd1*(C0_L2 - C0_R) + Kd1*Kd2

    c = -Kd1 * Kd2 * C0_R

    d = tt.sqrt(a*a - 3*b)

    e = tt.clip((-2. * a**3 + 9. * a * b - 27. * c) / (2. * d**3), -1, 1)

    theta = tt.arccos(e)

    RL1 = C0_L1 * (2. * d * tt.cos(theta/3.) - a) / (3. * Kd1 + (2. * d * tt.cos(theta/3.) - a))
    RL2 = C0_L2 * (2. * d * tt.cos(theta/3.) - a) / (3. * Kd2 + (2. * d * tt.cos(theta/3.) - a))

    return RL1, RL2


def heats_RacemicMixtureBindingModel(V0, DeltaVn, P0, Ls, rho, DeltaH1, DeltaH2, DeltaH_0,
                                     DeltaG1, DeltaDeltaG,
                                     beta, N):
    """
    Expected heats of injection for racemic mixtrue binding model.
    :param V0: cell volume (liter)
    :param DeltaVn: injection volumes (liter)
    :param P0: Cell concentration (millimolar)
    :param Ls: Syringe concentration (millimolar)
    :param rho: ratio between concentration of ligand1 and the total concentration of the syringe
    :param DeltaH1: enthalpies of binding of ligand1 (kcal/mol)
    :param DeltaH2: enthalpies of binding of ligand2 (kcal/mol)
    :param DeltaH_0: heat of injection (cal)
    :param DeltaG1: free energy of binding of ligand1 (kcal/mol)
    :param DeltaDeltaG: difference in binding free energy between ligand2 and ligand1: 
                        DeltaDeltaG = DeltaG2 - DeltaG1 > 0. DeltaDeltaG is always positive
    :param beta: inverse temperature * gas constant (mole / kcal)
    :param N: number of injections

    :return: q_n - expected injection heat (calorie)
    """

    # compute desociation constant (M)
    DeltaG2 = DeltaG1 + DeltaDeltaG

    Kd1 = tt.exp(beta * DeltaG1)   # dissociation constant of ligand1 (M)
    Kd2 = tt.exp(beta * DeltaG2)   # dissociation constant of ligand2 (M)

    # Compute complex concentrations in the sample after injection n.
    RL1n = tt.zeros([N])
    RL2n = tt.zeros([N])
    dcum = 1.0  # cumulative dilution factor (dimensionless)
    for n in range(N):
        # Instantaneous injection model (perfusion)
        # dilution factor for this injection (dimensionless)
        d = 1.0 - (DeltaVn[n] / V0)
        dcum *= d  # cumulative dilution factor

        # total concentration of protein in sample cell after n injections (mol)
        C0_R = P0 * 1.e-3 * dcum

        # total concentration of ligands in sample cell after n injections (mol)
        C0_L = Ls * 1.e-3 * (1. - dcum)

        # total concentration of ligand 1
        C0_L1 = rho * C0_L
        # total concentration of ligand 2
        C0_L2 = (1. - rho) * C0_L

        RL1n_n, RL2n_n = _equilibrium_concentrations(Kd1, Kd2, C0_R, C0_L1, C0_L2, V0)
        RL1n = tt.set_subtensor(RL1n[n], RL1n_n)
        RL2n = tt.set_subtensor(RL2n[n], RL2n_n)

    # Compute expected injection heats.
    # q_n_model[n] is the expected heat from injection n
    q_n = tt.zeros([N])

    # Instantaneous injection model (perfusion)
    # first injection
    #q_n[0] = (DeltaH1 * V0 * RL1n[0] + DeltaH2 * V0 * RL2n[0]) * 1000 + DeltaH_0
    q_n = tt.set_subtensor(q_n[0], (DeltaH1 * V0 * RL1n[0] + DeltaH2 * V0 * RL2n[0]) * 1000 + DeltaH_0)

    for n in range(1, N):
        d = 1.0 - (DeltaVn[n] / V0)  # dilution factor (dimensionless)
        # subsequent injections
        q_n = tt.set_subtensor(q_n[n], 
                (DeltaH1 * V0 * (RL1n[n] - d*RL1n[n-1]) + DeltaH2 * V0 * (RL2n[n] - d*RL2n[n-1])) * 1000 + DeltaH_0)

    return q_n


def logsigma_guesses(q_n_cal):
    log_sigma_guess = np.log(q_n_cal[-4:].std())
    log_sigma_min = log_sigma_guess - 10
    log_sigma_max = log_sigma_guess + 5
    return log_sigma_min, log_sigma_max


def deltaH0_guesses(q_n_cal):
    heat_interval = (q_n_cal.max() - q_n_cal.min())
    DeltaH_0_min = q_n_cal.min() - heat_interval
    DeltaH_0_max = q_n_cal.max() + heat_interval
    return DeltaH_0_min, DeltaH_0_max


def lognormal_prior(name, stated_value, uncertainty):
    """
    Define a pymc3 prior for a deimensionless quantity
    :param name: str
    :param stated_value: float
    :uncertainty: float
    :rerurn: pymc3.Lognormal
    """
    m = stated_value
    v = uncertainty ** 2
    return pymc3.Lognormal(name,
                           mu=tt.log(m / tt.sqrt(1 + (v / (m ** 2)))),
                           tau=1.0 / tt.log(1 + (v / (m ** 2))),
                           testval=m)


def uniform_prior(name, lower, upper):
    """
    :param name: str
    :param lower: float
    :param upper: float
    :return: pymc3.Uniform
    """
    return pymc3.Uniform(name, lower=lower, upper=upper)


def make_RacemicMixtureBindingModel(q_actual_cal,
                                    injection_volumes, 
                                    cell_concentration, syringe_concentration,
                                    cell_volume=0.001434,
                                    temperature=298.15,
                                    dcell=0.1, dsyringe=0.1,
                                    uniform_P0=False, P0_min=None, P0_max=None, 
                                    uniform_Ls=False, Ls_min=None, Ls_max=None,
                                    is_rho_free_param=False):
    """
    to create a pymc3 model
    
    :param q_actual_cal: observed heats in calorie, array-like
    :param injection_volumes: injection volumes in liter, array-like
    :param cell_concentration: concentration of the sample cell in milli molar, float
    :param syringe_concentration: concentration of the syringe in milli molar, float
    :param cell_volume: volume of sample cell in liter, float
    :param temperature: temprature in kelvin, float
    :param dcell: relative uncertainty in cell concentration, float
    :param dsyringe: relative uncertainty in syringe concentration, float
    :param uniform_P0: if True, use uniform prior for cell concentration, bool
    :param P0_min: only use if uniform_P0 is True, float
    :param P0_max: only use if uniform_P0 is True, float
    :param uniform_Ls: if True, use uniform prior for syringe concentration, bool
    :param Ls_min: only use if uniform_Ls is True, float
    :param Ls_max: only use if uniform_Ls is True, float
    :param is_rho_free_param: bool
    :return: an instance of pymc3.model.Model
    """
    assert len(q_actual_cal) == len(injection_volumes), "q_actual_cal and injection_volumes must have the same len."
    
    if uniform_P0 and (P0_min is None or P0_max is None):
        raise ValueError("If uniform_P0 is True, both P0_min and P0_max must be provided")
    
    if uniform_Ls and (Ls_min is None or Ls_max is None):
        raise ValueError("If uniform_Ls is True, both Ls_min and Ls_max must be provided")

    if is_rho_free_param:
        print("EnantiomerBindingModel")
    else:
        print("RacemicMixtureBindingModel")

    V0 = cell_volume

    DeltaVn = injection_volumes

    beta = 1 / KB / temperature
    n_injections = len(q_actual_cal)

    DeltaH_0_min, DeltaH_0_max = deltaH0_guesses(q_actual_cal)
    log_sigma_min, log_sigma_max = logsigma_guesses(q_actual_cal)

    stated_P0 = cell_concentration
    print("stated_P0", stated_P0)
    uncertainty_P0 = dcell * stated_P0
    
    stated_Ls = syringe_concentration
    print("stated_Ls", stated_Ls)
    uncertainty_Ls = dsyringe * stated_Ls

    with pymc3.Model() as model:

        # prior for receptor concentration
        if uniform_P0:
            print("Uniform prior for P0")
            P0 = uniform_prior("P0", lower=P0_min, upper=P0_max)
        else:
            print("LogNormal prior for P0")
            P0 = lognormal_prior("P0", stated_value=stated_P0, uncertainty=uncertainty_P0)

        # prior for ligand concentration
        if uniform_Ls:
            print("Uniform prior for Ls")
            Ls = uniform_prior("Ls", lower=Ls_min, upper=Ls_max)
        else:
            print("LogNormal prior for Ls")
            Ls = lognormal_prior("Ls", stated_value=stated_Ls, uncertainty=uncertainty_Ls)

        if is_rho_free_param:
            rho = uniform_prior("rho", lower=0., upper=1.)
        else:
            rho = 0.5

        # prior for DeltaG1, and DeltaDeltaG
        DeltaG1 = uniform_prior("DeltaG1", lower=-40., upper=40.)
        DeltaDeltaG = uniform_prior("DeltaDeltaG", lower=0., upper=40.)

        # prior for DeltaH1 and DeltaH2
        DeltaH1 = uniform_prior("DeltaH1", lower=-100., upper=100.)
        DeltaH2 = uniform_prior("DeltaH2", lower=-100., upper=100.)

        # prior for DeltaH_0
        DeltaH_0 = uniform_prior("DeltaH_0", lower=DeltaH_0_min, upper=DeltaH_0_max)

        # prior for log_sigma
        log_sigma = uniform_prior("log_sigma", lower=log_sigma_min, upper=log_sigma_max)

        q_model_cal = heats_RacemicMixtureBindingModel(V0, DeltaVn, P0, Ls, rho,
                                                       DeltaH1, DeltaH2, DeltaH_0, DeltaG1, DeltaDeltaG,
                                                       beta, n_injections)

        sigma_cal = tt.exp(log_sigma)

        q_obs = pymc3.Normal("q_obs", mu=q_model_cal, sd=sigma_cal, observed=q_actual_cal)

    return model


In [6]:
def load_heat_micro_cal(origin_heat_file):
    """
    :param origin_heat_file: str, name of heat file
    :return: 1d ndarray, heats in micro calorie
    """

    heats = []
    with open(origin_heat_file) as handle:
        handle.readline()
        for line in handle:
            if len(line.split()) == 6:
                heats.append(np.float(line.split()[0]))

    return np.array(heats)


def extract_samples_from_trace(trace, thin=1, burn=0):
    params = {var: trace.get_values(var, thin=thin, burn=burn) for var in trace.varnames}
    return params